In [6]:
import time
from pipelineapi import PipelineAPI

def get_dataset(anns_dir, data_pipeline_api, dataset_names):
    task, file_name = data_pipeline_api.data_file(video_source_names=[], dataset_names=dataset_names,
                categories_names=[], label_statuses=[], bulk_size=6000)
    print(task) ; print(file_name)

    while not os.path.exists(os.path.join(anns_dir, os.path.splitext(file_name)[0]+'_1.json')):
        
        print(data_pipeline_api.get_file(task_id=task, folder=anns_dir))
        time.sleep(30) 

In [8]:
import os 
anns_dir = '/media/dean/datastore/datapipeline_api_dir/all_frames/'
data_pipeline_api =  PipelineAPI("http://ec2-54-184-229-238.us-west-2.compute.amazonaws.com/", "admin@pronto.ai", "admin123!")
dataset_names = ['bdd_set', 'kache_set', 'holdout_set', 'china_set', 'pronto_set']
get_dataset(anns_dir, data_pipeline_api, dataset_names)

2acc1ffb-f6a3-420e-9f43-6026003246ce
20a4e97c-256d-42b8-937f-ec18600165a6.zip
/media/dean/datastore/datapipeline_api_dir/all_frames/
IN PROCESS
/media/dean/datastore/datapipeline_api_dir/all_frames/
IN PROCESS
/media/dean/datastore/datapipeline_api_dir/all_frames/
None
/media/dean/datastore/datapipeline_api_dir/all_frames/
None
/media/dean/datastore/datapipeline_api_dir/all_frames/
None
/media/dean/datastore/datapipeline_api_dir/all_frames/
None
/media/dean/datastore/datapipeline_api_dir/all_frames/
None
/media/dean/datastore/datapipeline_api_dir/all_frames/
None
/media/dean/datastore/datapipeline_api_dir/all_frames/
None
/media/dean/datastore/datapipeline_api_dir/all_frames/
None
/media/dean/datastore/datapipeline_api_dir/all_frames/
None
/media/dean/datastore/datapipeline_api_dir/all_frames/
None
/media/dean/datastore/datapipeline_api_dir/all_frames/
None
/media/dean/datastore/datapipeline_api_dir/all_frames/
None
/media/dean/datastore/datapipeline_api_dir/all_frames/
None
/media/dea

In [10]:
import glob

set_batches = glob.glob(os.path.join(anns_dir, '*.json'))

for set_batch in set_batches:
    with open(set_batch, 'r') as f:
        trn_set = json.load(f)
        for idx, img_data in enumerate(trn_set):
            img_data['labels'] = []
    with open(set_batch, 'w') as f:
        json.dump(trn_set, f)
    

In [1]:
import urllib
import http
from functools import wraps
import ntpath
from tqdm import tqdm
import ast
import socket
from pipelineapi import PipelineAPI
import toml
import os
import json
from subprocess import Popen,PIPE,STDOUT,call



def path_leaf(path):
    if urllib.parse.urlparse(path).scheme != "" or os.path.isabs(path):
        path = os.path.split(path)[-1]

    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

In [ ]:
import glob
import copy

anns_dir = '/media/dean/datastore/datapipeline_api_dir/all_frames/'
scalabel_anns_dir = '/media/dean/datastore/datapipeline_api_dir/all_frames/scalabel/'
set_batches = glob.glob(os.path.join(anns_dir, '*.json'))
scalabel_batches = glob.glob(os.path.join(scalabel_anns_dir, '*.json'))
scalabel_dir = os.path.join(scalabel_anns_dir, 'scalabel_anns')
os.makedirs(scalabel_dir, exist_ok = True)

for scalabel_batch in scalabel_batches:
    new_scalabel_batch = os.path.join(scalabel_dir, path_leaf(scalabel_batch))
    with open(scalabel_batch, 'r') as f:
        scalabel_set = json.load(f)
    new_set = []    
    for idx, scalabel_img_data in enumerate(scalabel_set):
        # find labels from corresponding set_files
        found_frame = False
    
    
        for set_batch in set_batches:
            if not found_frame:
                with open(set_batch, 'r') as f:
                    trn_set = json.load(f)
                    for idx, img_data in enumerate(trn_set):
                        # check for match
                        if scalabel_img_data['url'] == img_data['url']:
                            found_frame = True
                            if scalabel_img_data.get('labels', None):
                                for lbl in scalabel_img_data['labels']:
                                    lbl['kache_label_id'] = None
                                    lbl['scalabel_label_id'] = lbl['id']

                                img_data['labels'] = copy.deepcopy(scalabel_img_data['labels'])
                                new_set.append(copy.deepcopy(img_data))
                                break
            else:
                break




        # img_data['labels'] = []
    with open(new_scalabel_batch, 'w') as f:
        json.dump(new_set, f)
    

In [ ]:
from collections import defaultdict

kache_dir = os.path.join(scalabel_dir, 'kache_sets')
os.makedirs(kache_dir, exist_ok = True)

kache_batches = glob.glob(os.path.join(scalabel_dir, '*.json'))
for kache_batch in kache_batches:
    dsets = defaultdict(list)
    with open(kache_batch, 'r') as f:
        kache_set = json.load(f)
    for idx, img_data in enumerate(kache_set):
        dsets[img_data['dataset_name']].append(copy.deepcopy(img_data))
        
    for dataset_name, dataset_data in dsets.items():
        new_kache_set = os.path.join(kache_dir, os.path.splitext(path_leaf(kache_batch))[0]+'_'+dataset_name+os.path.splitext(path_leaf(kache_batch))[-1])
        with open(new_kache_set, 'w') as f:
            json.dump(dataset_data, f)
        
            